# Stackelberg vs Observing Stackelberg

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import sys
sys.path.append('../')

In [3]:
import source.players as pl
import source.parsers as parsers
import source.game as game
import source.player as player
import source.environment as environment
import source.errors as errors
import source.players.attackers as attackers
import source.players.base_defenders as base_defenders
import source.players.defenders as defenders
import source.runner as runner
import source.players.holmes as holmes
import source.players.sad as sad
import source.players.b2bw2w as b2bw2w
import source.util as util

import concurrent.futures
import pkgutil
import logging
import sys
from source.util import *
import numpy as np
import pandas as pd
from importlib import *
import time
import os
import re
from copy import deepcopy, copy
import datetime
from contextlib import redirect_stdout

In [4]:
# run once

lvl = logging.DEBUG
logger = logging.getLogger("source")
logger.setLevel(lvl)
#ch = logging.StreamHandler(sys.stdout)
ch = logging.FileHandler("status", mode='w')
ch.setLevel(lvl)
#formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
formatter = logging.Formatter('%(message)s')
ch.setFormatter(formatter)
logger.addHandler(ch)

In [5]:
lvl = logging.WARNING
logger.setLevel(lvl)
ch.setLevel(lvl)

In [6]:
# if you change something in the files, don't forget to reload
reload(player)
reload(base_defenders)
reload(attackers)
reload(defenders)
reload(errors)
reload(environment)
reload(runner)
reload(parsers)
reload(game)
reload(holmes)
reload(sad)
reload(b2bw2w)
reload(util)

<module 'source.util' from '..\\source\\util.py'>

In [7]:
time_horizon = 1000
T = 5
#targets = util.gen_targets(T, low=1, high=10)
targets = [10, 2, 2, 2, 2]
values = tuple((v, v) for v in targets)
#obs = gen_observabilities_correlated_with_values(values)
g = game.Game(values, time_horizon)
g.attackers = [1]
g.defenders = [0]
a = [attackers.StackelbergAttacker(g, 1)]
d = [base_defenders.StackelbergDefender(g, 0)]
p = [attackers.ObservingStackelbergAttacker(g, 1, 1), attackers.StackelbergAttacker(g, 1)]
g.set_players(d, a, p)

In [8]:
d[0].br_stackelberg()
print(d[0].br_stackelberg_strategy)

Academic license - for non-commercial use only
[0.8095238095238093, 0.047619047619047616, 0.04761904761904767, 0.04761904761904767, 0.04761904761904767]


In [9]:
obs = {0: 1, 1: 0.1, 2: 0.1, 3: 0.1, 4: 0.1}
obs_vect = [obs.get(i) for i in range(len(targets))]
go1 = game.GameWithObservabilities(values, time_horizon)
go1.attackers = [1]
go1.defenders = [0]
a1 = [attackers.ObservingStackelbergAttacker(go1, 1)]
d1 = [base_defenders.ObservingStackelbergDefender(go1, 0)]
p1 = [attackers.StackelbergAttacker(go1, 1), attackers.ObservingStackelbergAttacker(go1, 1)]
go1.set_players(d1, a1, p1)
go1.set_observabilities(obs)

In [10]:
d1[0].br_stackelberg()
print(d1[0].br_stackelberg_strategy)

[0.8009950248756219, 0.049751243781093954, 0.04975124378109552, 0.049751243781094315, 0.049751243781094315]


In [11]:
e = runner.Experiment(go1)

In [15]:
e.run_interaction_with_observations()

In [16]:
print(e.game.strategy_history)
print(e.game.history)
print(e.game.observation_history)
print(e.environment.last_exp_loss)
print(e.environment.last_opt_loss)

[{0: [0.8009950248756219, 0.049751243781093954, 0.04975124378109552, 0.049751243781094315, 0.049751243781094315], 1: [0, 1, 0, 0, 0]}, {0: [0.8009950248756219, 0.049751243781093954, 0.04975124378109552, 0.049751243781094315, 0.049751243781094315], 1: [0, 1, 0, 0, 0]}, {0: [0.8009950248756219, 0.049751243781093954, 0.04975124378109552, 0.049751243781094315, 0.049751243781094315], 1: [0, 1, 0, 0, 0]}, {0: [0.8009950248756219, 0.049751243781093954, 0.04975124378109552, 0.049751243781094315, 0.049751243781094315], 1: [0, 1, 0, 0, 0]}]
[{1: [1], 0: [0]}, {1: [1], 0: [0]}, {1: [1], 0: [0]}, {1: [1], 0: [4]}]
[{0: 1, 1: 0, 2: 0, 3: 0, 4: 0}, {0: 1, 1: 0, 2: 0, 3: 0, 4: 0}, {0: 1, 1: 0, 2: 0, 3: 0, 4: 0}, {0: 1, 1: 0, 2: 0, 3: 0, 4: 0}]
1.900497512437812
1.904761904761905


In [20]:
go1.history[-1][1][0]

1

In [22]:
a1[0].last_strategy[1]

1

In [26]:
from math import log

In [27]:
log(1)

0.0